In [135]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az
from scipy import stats

In [193]:
# 1

def generate_clients_nr():
    clients_nr = stats.poisson.rvs(mu=20, size=10000)
    clients_nr = [i for i in clients_nr if i != 0]
    return clients_nr
    
def generate_requests(clients_nr):
    requests = []
    for nr in clients_nr:    
        requests.append(np.array([abs(i) for i in stats.norm.rvs(1, 0.5, size=nr)]))
    return requests

def generate_cooks(clients_nr, alpha):
    cooks = []
    for nr in clients_nr:    
        cooks.append(stats.expon.rvs(scale=1/alpha, size=nr))
    return cooks

In [194]:
alpha = 0.01
clients_nr = generate_clients_nr()
requests = generate_requests(clients_nr=clients_nr)
cooks = generate_cooks(clients_nr=clients_nr, alpha=alpha)

In [195]:
# 2

clients_serving_time_less = 0
p = 0
found = False
max_alpha = alpha
best_prob = 0

while True:
    for iteration in range(len(clients_nr)): 
        for client in range(clients_nr[iteration]): 
            if requests[iteration][client] + cooks[iteration][client] < 15:
                clients_serving_time_less += 1
        p = p + (clients_serving_time_less / clients_nr[iteration])
        clients_serving_time_less = 0
    # calculating the mean for the probability
    p = p / len(clients_nr)
    print(f"The probability obtained with alpha = {alpha} is {p}")
    
    # maximum alpha is found
    if found and abs(p - 0.95) > 0.001:
        print()
        print("-----Solution------")
        print(f"""The maximum alpha for getting the probability of serving all the clients less than in 15 min is: 
            {alpha} and the probability obtained is: {best_prob}""")
        break
    
    # a good alpha is found
    if abs(p - 0.95) < 0.001:
        found = True
        max_alpha = alpha
        best_prob = p

    # take bigger steps if far from solution
    if not found and abs(p - 0.95) > 0.04:
        alpha += 0.025
    else:
        alpha += 0.001
    p = 0
    cooks = generate_cooks(clients_nr=clients_nr, alpha=alpha)
    

The probability obtained with alpha = 0.01 is 0.13007286264941578
The probability obtained with alpha = 0.035 is 0.3894352893713689
The probability obtained with alpha = 0.060000000000000005 is 0.5679946802396382
The probability obtained with alpha = 0.085 is 0.6953586070857989
The probability obtained with alpha = 0.11000000000000001 is 0.7863735411010282
The probability obtained with alpha = 0.135 is 0.8483487162724949
The probability obtained with alpha = 0.16 is 0.8923430608976831
The probability obtained with alpha = 0.185 is 0.9248137867881229
The probability obtained with alpha = 0.186 is 0.9261570178955986
The probability obtained with alpha = 0.187 is 0.9262448750953644
The probability obtained with alpha = 0.188 is 0.9276865718172623
The probability obtained with alpha = 0.189 is 0.9296827318405805
The probability obtained with alpha = 0.19 is 0.9301491693952532
The probability obtained with alpha = 0.191 is 0.9307842481138078
The probability obtained with alpha = 0.192 is 0.

In [199]:
# 3
# how much will the client wait to get the food

alpha = max_alpha
cooking_time = 0
clients_nr = generate_clients_nr()
cooks = generate_cooks(clients_nr=clients_nr, alpha=alpha)

for iteration in range(len(clients_nr)): 
        cooking_time += sum(cooks[iteration])
        
cooking_time = cooking_time / sum(clients_nr)
print(f"The average cooking time obtained with alpha = {alpha} is {cooking_time}")


The average cooking time obtained with alpha = 0.21500000000000002 is 4.6575091178489245
